PROJETO GESTÃO E ANÁLISE DE ESTAÇÕES DE CARREGAMENTO DE CARROS ELÉTRICOS:

O projeto consiste em importar, tratar e analisar dados de estações elétricas e veículos, com visualizações em Power BI.
Etapas:

1-Importação: carregar dados do Excel para Python.

2-Tratamento: limpar nulos, padronizar colunas e tipos.

3-SQL: criar tabelas e inserir dados mantendo integridade.

3-Análise: consultas SQL para insights.

4-Dashboard: criar visualizações interativas no Power BI.

In [ ]:
#BIBLIOTECAS NECESSARIAS:
""" pip install pandas
pip install --upgrade pandas
pip install pyodbc """

In [130]:
#IMPORTANDO BIBLIOTECAS
import pandas as pd
import pyodbc 

IMPORTANDO ARQUIVOS EXCEL PARA O VSCODE E TRANSFORMANDO EM DATAFRAMES

In [131]:
df_estacoes=pd.read_excel(r"C:\Users\muril\Downloads\Arquivos_Base_estacoes_eletricas\arquivos_base_estacoes\charging_stations_2025_world.xlsx")
df_paises_infos=pd.read_excel(r"C:\Users\muril\Downloads\Arquivos_Base_estacoes_eletricas\arquivos_base_estacoes\country_summary_2025.xlsx")
df_veiculo=pd.read_excel(r"C:\Users\muril\Downloads\Arquivos_Base_estacoes_eletricas\arquivos_base_estacoes\ev_models_2025.xlsx")
df_paises=pd.read_excel(r"C:\Users\muril\Downloads\Arquivos_Base_estacoes_eletricas\arquivos_base_estacoes\world_summary_2025.xlsx")

In [146]:
df_estacoes.head(4)

,id_estacao,nome_local,cidade,sigla,estado,latitude,longitude,qtd_portas,potencia,potencia_categoria,carregamento_rapido,id
0,307660,Av. de Tarragona,Andorra,AD,NaN,42505254.0,1528861.0,10.0,300.0,DC_ULTRA_(>=150kW),Carregamento rapido,53
1,301207,Parquing Costa Rodona,Encamp,AD,NaN,42537213.0,1727014.0,10.0,22.0,AC_HIGH_(22-49kW),Carregamento lento,53
2,301206,Hotel Naudi,NaN,AD,NaN,42576811.0,1666061.0,1.0,11.0,AC_L2_(7.5-21kW),Carregamento lento,53
3,301205,Hotel Piolets Soldeu Centre,NaN,AD,NaN,42576466.0,1667317.0,1.0,22.0,AC_HIGH_(22-49kW),Carregamento lento,53


In [147]:
df_paises_infos.head(4)

,id,sigla,qtd_estacoes
0,1,US,82138
1,2,UK,26825
2,3,DE,23373
3,4,ES,17825


In [148]:
df_veiculo.head(4)

,fabricante,modelo,regiao_mercado,tipo_motor,first_year,carro_categoria,sigla,id,country_code
0,Tesla,Model S,Global (US/EU/UK/ME),BEV,2012.0,Sedan,US,1,US
1,Tesla,Model 3,Global (US/EU/UK/ME/CN),BEV,2017.0,Sedan,US,1,US
2,Tesla,Model X,Global (US/EU/UK/ME),BEV,2015.0,SUV,US,1,US
3,Tesla,Model Y,Global (US/EU/UK/ME/CN),BEV,2020.0,SUV,US,1,US


In [149]:
df_paises.head(5)

,sigla,pais,qtd_estacoes,potencia_max_estacao,id
0,US,United States,83821,600.0,1
1,UK,United Kingdom,27437,560.0,2
2,DE,Germany,24465,400.0,3
3,CA,Canada,18203,400.0,5
4,ES,Spain,17864,1000.0,4


Adicionando uma coluna de id no df_paises infos e ligando com os outros DE's

In [136]:
# 1) Criar coluna id no df_paises_infos (country_summary)
df_paises_infos = df_paises_infos.reset_index(drop=True)  # garante índice limpo
df_paises_infos.insert(0, "id", range(1, len(df_paises_infos) + 1))  # cria ID sequencial

# 2) Reorganizar colunas (id primeiro, o resto depois)
cols_paises_infos = ['id'] + [col for col in df_paises_infos.columns if col != 'id']
df_paises_infos = df_paises_infos[cols_paises_infos]


In [137]:

# ========================================
# 2) Fazer os merges para herdar o id_pais
# ========================================

# --- Stations ---
df_estacoes = df_estacoes.merge(
    df_paises_infos[["id", "country_code"]],
    on="country_code",
    how="left"
)

# --- World Summary ---
df_paises = df_paises.merge(
    df_paises_infos[["id", "country_code"]],
    on="country_code",
    how="left"
)

# --- EV Models ---
df_veiculo = df_veiculo.merge(
    df_paises_infos[["id", "country_code"]],
    left_on="origin_country",
    right_on="country_code",
    how="left"
)


RENOMEANDO/TRADUZINDO COLUNAS DE TODOS OS DATAFRAMES

In [142]:
df_estacoes=df_estacoes.rename(columns={
        'id_x':'id_estacao',
        'name':'nome_local',
        'city':'cidade',
        'country_code':'sigla',
        'state_province':'estado',
        'ports':'qtd_portas',
        'power_kw':'potencia',
        'power_class':'potencia_categoria',
        'is_fast_dc':'carregamento_rapido',
        'id_y':'id'
    })

In [143]:
df_paises_infos=df_paises_infos.rename(columns={
        'id':'id',
        'country_code':'sigla',
        'stations':'qtd_estacoes'
    })

In [144]:
df_veiculo=df_veiculo.rename(columns={
        'make':'fabricante',
        'model':'modelo',
        'market_regions':'regiao_mercado',
        'powertrain':'tipo_motor',
        'body_style':'carro_categoria',
        'id':'id',
        'origin_country':'sigla'
    })

In [145]:
df_paises=df_paises.rename(columns={
        'country_code':'sigla',
        'country':'pais',
        'count':'qtd_estacoes',
        'max_power_kw_max':'potencia_max_estacao',
        'id':'id'
    })

VERIFICANDO A QUANTIDA DE NULOS EM CADA COLUNA DE CADA DATAFRAME

In [155]:
df_estacoes.isna().sum()

id_estacao             0
nome_local             0
cidade                 0
sigla                  0
estado                 0
latitude               0
longitude              0
qtd_portas             0
potencia               0
potencia_categoria     0
carregamento_rapido    0
id                     0
dtype: int64

In [156]:
df_paises_infos.isna().sum()

id              0
sigla           0
qtd_estacoes    0
dtype: int64

In [179]:
df_veiculo.isna().sum()

id_veiculo         0
fabricante         0
modelo             0
regiao_mercado     0
tipo_motor         0
first_year         0
carro_categoria    1
sigla              0
id                 0
country_code       0
dtype: int64

In [158]:
df_paises.isna().sum()

sigla                   0
pais                    0
qtd_estacoes            0
potencia_max_estacao    0
id                      0
dtype: int64

TRATANDO NULOS DE TODAS OS DATAFRAMES

In [154]:
#tratando nulos df_estacoes
df_estacoes['cidade']=df_estacoes['cidade'].fillna('nao informado')
df_estacoes['estado']=df_estacoes['estado'].fillna('nao informado')
df_estacoes['sigla']=df_estacoes['sigla'].fillna('nao informado')
df_estacoes['latitude']=df_estacoes['latitude'].fillna(0)
df_estacoes['longitude']=df_estacoes['longitude'].fillna(0)
df_estacoes['qtd_portas']=df_estacoes['qtd_portas'].fillna(4)
df_estacoes['potencia']=df_estacoes['potencia'].fillna(df_estacoes['potencia'].mean().round(2))
df_estacoes['potencia_categoria']=df_estacoes['potencia_categoria'].fillna("nao informado")
df_estacoes['carregamento_rapido']=df_estacoes['carregamento_rapido'].fillna("nao informado")
df_estacoes['id']=df_estacoes['id'].fillna(1)



#tratando nulos df_paises_infos
df_paises_infos['sigla']=df_paises_infos['sigla'].fillna('nao informado')

#tratando nulos df_veiculos
df_veiculo['tipo_motor']=df_veiculo['tipo_motor'].fillna('nao informado')
df_veiculo['first_year']=df_veiculo['first_year'].fillna(df_veiculo['first_year'].mean().round(0))
df_veiculo['carro_categoria']=df_veiculo['carro_categoria'].fillna('nao informado')

#tratando nulos df_pais
df_paises['sigla']=df_paises['sigla'].fillna('nao informado')
df_paises['potencia_max_estacao']=df_paises['potencia_max_estacao'].fillna(df_paises['potencia_max_estacao'].median().round(2))


TRATANDO CATEGORIAS EXISTENTES NA TABELA DE VEICULOS

In [159]:
df_estacoes['potencia_categoria'].unique()

array(['DC_ULTRA_(>=150kW)', 'AC_HIGH_(22-49kW)', 'AC_L2_(7.5-21kW)',
       'AC_L1_(<7.5kW)', 'DC_FAST_(50-149kW)', 'UNKNOWN', 'nao informado'],
      dtype=object)

In [160]:
# Dicionário com as traduções
traducao_tipo_corrente = {
    'AC_L1_(<7.5kW)': 'AC',
    'AC_L2_(7.5-21kW)': 'AC',
    'AC_HIGH_(22-49kW)': 'AC',
    'DC_FAST_(50-149kW)': 'DC',
    'DC_ULTRA_(>=150kW)': 'DC'
}

traducao_potencia_categoria = {
    'AC_L1_(<7.5kW)': 'Baixa Potência',
    'AC_L2_(7.5-21kW)': 'Média Potência',
    'AC_HIGH_(22-49kW)': 'Alta Potência',
    'DC_FAST_(50-149kW)': 'Rápida',
    'DC_ULTRA_(>=150kW)': 'Ultrarrápida'
}

df_estacoes['tipo_corrente'] = df_estacoes['potencia_categoria'].map(traducao_tipo_corrente)
df_estacoes['potencia_categoria_traduzida'] = df_estacoes['potencia_categoria'].map(traducao_potencia_categoria)

df_estacoes['tipo_corrente']=df_estacoes['tipo_corrente'].fillna('não informado')
df_estacoes['potencia_categoria_traduzida']=df_estacoes['potencia_categoria_traduzida'].fillna('não informado')

df_estacoes.head(3)

,id_estacao,nome_local,cidade,sigla,estado,latitude,longitude,qtd_portas,potencia,potencia_categoria,carregamento_rapido,id,tipo_corrente,potencia_categoria_traduzida
0,307660,Av. de Tarragona,Andorra,AD,nao informado,42505254.0,1528861.0,10.0,300.0,DC_ULTRA_(>=150kW),Carregamento rapido,53,DC,Ultrarrápida
1,301207,Parquing Costa Rodona,Encamp,AD,nao informado,42537213.0,1727014.0,10.0,22.0,AC_HIGH_(22-49kW),Carregamento lento,53,AC,Alta Potência
2,301206,Hotel Naudi,nao informado,AD,nao informado,42576811.0,1666061.0,1.0,11.0,AC_L2_(7.5-21kW),Carregamento lento,53,AC,Média Potência


In [161]:
df_veiculo['carro_categoria'].unique()

array(['Sedan', 'SUV', 'Pickup', 'Hatchback', 'nao informado',
       'Crossover', 'Van', 'Sedan/Wagon', 'SUV (3-row)', 'Sedan/SUV',
       'Liftback', 'City car'], dtype=object)

In [162]:
map_body_style = {
    'Sedan': 'Sedan',
    'SUV': 'SUV',
    'Hatchback': 'Hatch',
    'Van': 'Van',
    'Sedan/SUV': 'SUV',
    'Pickup': 'Van',       # pode jogar pickup como "Van" ou "Outros"
    'SUV (3-row)': 'SUV',      # coupe pode ser simplificado como sedan
    'Sedan/Wagon': 'Hatch',
    'Liftback': 'Hatch',
    'Crossover': 'SUV',
    'City car':'Hatch'
}

df_veiculo['id_veiculo'] = range(1, len(df_veiculo) + 1)

cols = ['id_veiculo'] + [col for col in df_veiculo.columns if col != 'id_veiculo']
df_veiculo = df_veiculo[cols]

df_veiculo['id_veiculo']=df_veiculo['id_veiculo'].fillna(0)

df_veiculo['carro_categoria'] = df_veiculo['carro_categoria'].map(map_body_style)
df_veiculo.head(3)

,id_veiculo,fabricante,modelo,regiao_mercado,tipo_motor,first_year,carro_categoria,sigla,id,country_code
0,1,Tesla,Model S,Global (US/EU/UK/ME),BEV,2012.0,Sedan,US,1,US
1,2,Tesla,Model 3,Global (US/EU/UK/ME/CN),BEV,2017.0,Sedan,US,1,US
2,3,Tesla,Model X,Global (US/EU/UK/ME),BEV,2015.0,SUV,US,1,US


REMOVENDO ID's DUPLICADOS

In [163]:
# Removendo duplicatas em cada tabela pela PK
df_paises = df_paises.drop_duplicates(subset=["sigla"])
df_paises_infos = df_paises_infos.drop_duplicates(subset=["id"])
df_veiculo = df_veiculo.drop_duplicates(subset=["id_veiculo"])
df_estacoes = df_estacoes.drop_duplicates(subset=["id_estacao"])

VERIFICANDO O TIPO DE CADA COLUNA DOS DATAFRAMES

In [164]:
df_estacoes.dtypes

id_estacao                        int64
nome_local                       object
cidade                           object
sigla                            object
estado                           object
latitude                        float64
longitude                       float64
qtd_portas                      float64
potencia                        float64
potencia_categoria               object
carregamento_rapido              object
id                                int64
tipo_corrente                    object
potencia_categoria_traduzida     object
dtype: object

In [165]:
df_paises_infos.dtypes

id               int64
sigla           object
qtd_estacoes     int64
dtype: object

In [178]:
df_veiculo.dtypes

id_veiculo          int64
fabricante         object
modelo             object
regiao_mercado     object
tipo_motor         object
first_year          int64
carro_categoria    object
sigla              object
id                  int64
country_code       object
dtype: object

In [167]:
df_paises.dtypes

sigla                    object
pais                     object
qtd_estacoes              int64
potencia_max_estacao    float64
id                        int64
dtype: object

TRANSFERINDO OS DADOS TRATADOS DE CADA DATA FRAME EM UM BANCO DE DADOS 

In [182]:

#Realizando a conexao com o banco de dados 
server = 'localhost'  # Nome do servidor SQL Server
database = 'estacoes_database_oficial'  #Nome do banco de dados

conexaoDB = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};'
    f'SERVER={server};'
    f'DATABASE={database};'
    'Trusted_Connection=yes;'
)

cursor = conexaoDB.cursor()   #Criando cursor de comando 

# --- APAGAR DADOS NA ORDEM CORRETA ---
cursor.execute('DELETE FROM estacoes')  # primeiro filhos
cursor.execute('DELETE FROM veiculos')  # primeiro filhos
cursor.execute('DELETE FROM paises')   # primeiro filhos
cursor.execute("DELETE FROM paises_infos") # depois pais

cursor.commit() #Validacao dos comandos executados

# --- INSERIR DADOS NA TABELA PAISES_INFOS ---
for index, linha in df_paises_infos.iterrows():
    cursor.execute("""
        INSERT INTO paises_infos (id,sigla,qtd_estacoes)
        VALUES (?, ?, ?)
    """, 
    linha.id, 
    linha.sigla,
    linha.qtd_estacoes 
    )

cursor.commit()

# --- INSERIR DADOS NA TABELA PAISES ---
for index, linha in df_paises.iterrows():
    cursor.execute("""
        INSERT INTO paises (sigla, pais, qtd_estacoes, potencia_max_estacao,id)
        VALUES (?,?,?,?,?)
    """, 
    linha.sigla, 
    linha.pais, 
    linha.qtd_estacoes, 
    linha.potencia_max_estacao,
    linha.id)

cursor.commit()

# --- ULTIMAS TRATATIVAS DO df_veiculo ANTES DE TRANSFERIR PARA UM BANCO DE DADOS ---
df_veiculo['first_year'] = df_veiculo['first_year'].fillna(0).astype(int)
df_veiculo = df_veiculo.where(pd.notnull(df_veiculo), None)
df_veiculo = df_veiculo[df_veiculo['id'].notna() & (df_veiculo['id'] != '')]

# --- INSERIR DADOS NA TABELA FILHA ---
for index, linha in df_veiculo.iterrows():
    cursor.execute("""
        INSERT INTO veiculos (
            id_veiculo, fabricante, modelo, regiao_mercado, tipo_motor,
            first_year, carro_categoria, sigla, id, country_code
        ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    """,
    linha['id_veiculo'],
    linha['fabricante'],
    linha['modelo'],
    linha['regiao_mercado'],
    linha['tipo_motor'],
    linha['first_year'],
    linha['carro_categoria'],
    linha['sigla'],
    linha['id'],
    linha['country_code']
    )

cursor.commit()

# --- INSERINDO DADOS NA TABELA ESTACAO ---
for index, linha in df_estacoes.iterrows():
    cursor.execute("""
        INSERT INTO estacoes (
            id_estacao, nome_local, cidade, sigla, estado,
            latitude, longitude, qtd_portas, potencia,
            potencia_categoria, carregamento_rapido,id,
            tipo_corrente, potencia_categoria_traduzida
        ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    """,
    linha['id_estacao'],
    linha['nome_local'],
    linha['cidade'],
    linha['sigla'],
    linha['estado'],
    linha['latitude'],
    linha['longitude'],
    linha['qtd_portas'],
    linha['potencia'],
    linha['potencia_categoria'],
    linha['carregamento_rapido'],
    linha['id'],
    linha['tipo_corrente'],
    linha['potencia_categoria_traduzida'])

cursor.commit()
cursor.close()
conexaoDB.close()
